In [254]:
import os, sys, csv

local_repo_path = 'D:\Palmer2020 Data Repo\Imagery datasets'
output_directory = 'D:\Palmer2020 Data Repo\Index_files'


expected_folders = ["DigitalGlobe products", "Drone data", "Landsat products", "Planet products"]

#glossaries (dictionaries) of abbreviations for each system
dg_gloss = {"GE01":"GeoEye-1", "QB02":"QuickBird-2", "WV01":"WorldView-1", "WV02":"WorldView-2", "WV03":"WorldView-3",
            "M":"B,G,R,NI", "P":"Panchromatic", "1BS":"geolocated, not orthorectified; relative radiance"}
ls_gloss = {"LM01":"Landsat-1 (MSS)", "LM03":"Landsat-3 (MSS)", "LM05":"Landsat-5 (MSS)", "LT04":"Landsat-4 (TM)",
            "LT05":"Landsat-5 (TM)", "LE07":"Landsat-7 (ETM+)", "LC08":"Landsat-8 (OLI)",
            "L1TP":"geolocated, orthorectified; TOA reflectance",
            "L1GT":"moderately geolocated, orthorectified; TOA reflectance",
            "L1GS":"poorly geolocated, not orthorectified; TOA reflectance",
            "L2SR":"geolocated, orthorectified; surface reflectance"}
re_gloss = {"RE2":"RapidEye-2", "RE4":"RapidEye-4", "RE5":"RapidEye-5"}

metadata_list = [["Year", "Month", "Day", "Hour", "Min", "Satellite", "Spectral Coverage", "Group", "Spatial Quality", "Corrections", "Filename"]]

def DG_handling(n,f):
    print("DG_handling")    
    for i in os.listdir("{l}\{f}".format(l=local_repo_path, f=f)):
        for j in os.listdir("{l}\{f}\{i}".format(l=local_repo_path, f=f, i=i)):
            if j.endswith(".tif"):
                #print(j)
                satellite = dg_gloss[j.split("_")[0]]
                spectr = dg_gloss[j.split("_")[3].split("-")[1][0]]
                corrections = dg_gloss[j.split("_")[3].split("-")[1][1:4]]
                date_code = j.split("_")[1]
                year, month, day, hour, minute, sec = date_code[0:4], date_code[4:6], date_code[6:8], date_code[8:10], date_code[10:12], date_code[12:14]
                metadata_list.append([year, month, day, hour, minute, satellite, spectr, expected_folders[n], "high resolution", corrections, j.split(".")[0]])

def Drone_handling(n,f):
    print("Drone images not included for now")
    
def LS_handling(n,f):
    print("LS_handling")
    for i in os.listdir("{l}\{f}".format(l=local_repo_path, f=f)):
        for j in os.listdir("{l}\{f}\{i}".format(l=local_repo_path, f=f, i=i)):
            #print(j)
            satellite = ls_gloss[j.split("_")[0]]
            date_code = j.split("_")[3]
            corrections = ls_gloss[j.split("_")[1]]
            year, month, day = date_code[0:4], date_code[4:6], date_code[6:8]
            metadata_list.append([year, month, day, "", "", satellite, "Multispectral", expected_folders[n], "medium resolution", corrections, j])

def Planet_handling(n,f):
    print("Planet_handling")
    for i in os.listdir("{l}\{f}".format(l=local_repo_path, f=f)):
        print(i)
        for j in os.listdir("{l}\{f}\{i}".format(l=local_repo_path, f=f, i=i)):
            if j.endswith(".tif") and "_udm" not in j:
                #print(j)
                if "PlanetScope" in i:
                    date_code = j.split("_")[2]
                    satellite = "PlanetScope"
                    spectr = "B,G,R,NI"
                elif "RapidEye" in i:
                    date_code = j.split("_")[1]
                    satellite = re_gloss[j.split("_")[2]]
                    spectr = "B,G,R,RE,NI"
                year, month, day = date_code.split("-")
                if j.split("_")[-2] == "SR":
                    corrections = "geolocated, orthorectified; surface reflectance"
                elif j.split("_")[-2] == "Visual":
                    corrections = "geolocated, orthorectified; color-corrected not reflectance"
                metadata_list.append([year, month, day, "", "", satellite, spectr, expected_folders[n], "high resolution", corrections, j.split(".")[0]])

In [255]:
for n,f in enumerate(os.listdir(local_repo_path)):
    if f == expected_folders[0]:
        print()
        DG_handling(n,f)
    if f == expected_folders[1]:
        print()
        Drone_handling(n,f)
    if f == expected_folders[2]:
        print()
        LS_handling(n,f)
    if f == expected_folders[3]:
        print()
        Planet_handling(n,f)
print(metadata_list)


DG_handling

Drone images not included for now

LS_handling

Planet_handling
PlanetScope [2016]
PlanetScope [2017]
PlanetScope [2018]
PlanetScope [2019]
PlanetScope [2020]
RapidEye [2009-2015]
[['Year', 'Month', 'Day', 'Hour', 'Min', 'Satellite', 'Spectral Coverage', 'Group', 'Spatial Quality', 'Corrections', 'Filename'], ['2010', '03', '02', '13', '20', 'GeoEye-1', 'B,G,R,NI', 'DigitalGlobe products', 'high resolution', 'geolocated, not orthorectified; relative radiance', 'GE01_20100302132032_1050410002BD1C00_10MAR02132032-M1BS-503931604060_01_P003_u08rf3031'], ['2010', '03', '02', '13', '20', 'GeoEye-1', 'Panchromatic', 'DigitalGlobe products', 'high resolution', 'geolocated, not orthorectified; relative radiance', 'GE01_20100302132032_1050410002BD1C00_10MAR02132032-P1BS-503931604060_01_P003_u08rf3031'], ['2010', '03', '02', '13', '20', 'GeoEye-1', 'B,G,R,NI', 'DigitalGlobe products', 'high resolution', 'geolocated, not orthorectified; relative radiance', 'GE01_20100302132035_105041

In [256]:
try:
  os.makedirs(output_directory)
except: pass

with open(output_directory + '\\satellite_index.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(metadata_list)